<a href="https://colab.research.google.com/drive/1fT11JsthwBe3mRlwqSwhrnlJpdTvEl6B?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

###	➕ What is Query Expansion/Transformation RAG?

Query Expansion is a technique used to improve the effectiveness of information retrieval systems by reformulating or augmenting the original query. The goal is to improve the recall of relevant documents by including related terms or concepts that might not have been explicitly mentioned in the original query.

### 🔧 Query Transformation RAG Implementation:

1. **Query Transformation:** We use the LLM to generate 3 alternative versions of the original query.
2. **Multi-Query Retrieval:** We retrieve documents using both the original query and the transformed queries.
3. **Deduplication:** We remove duplicate documents from the retrieved set.
4. **Response Generation:** Using the combined context from all retrieved documents, we generate a final response to the original query.

# ⚙️ Setup

1. **[LLM](https://groq.com/):** Groq's free Open source LLM endpoints([Groq API Key](https://console.groq.com/keys))
2. **[Vector Store](https://www.pinecone.io/learn/vector-database/):** [ChromaDB](https://www.trychroma.com/)
3. **[Embedding Model](https://qdrant.tech/articles/what-are-embeddings/):** [nomic-embed-text-v1.5](https://www.nomic.ai/blog/posts/nomic-embed-text-v1)
4. **[LLM Framework](https://python.langchain.com/v0.2/docs/introduction/):** LangChain
5. **[Huggingface API Key](https://huggingface.co/settings/tokens)**


# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1

# Import related libraries related to Langchain, HuggingfaceEmbedding

In [2]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import WebBaseLoader

In [3]:
import os
import getpass

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


### Provide Huggingface API Key. You can create Huggingface API key at following link

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


### Step 1: Load and preprocess data code

In [6]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    return chunks

### Step 2: Create multiple vector stores code

In [7]:
def create_vector_store(chunks):
  embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
  vectorstore = Chroma.from_documents(chunks, embeddings)
  return vectorstore

### Step 3: Query Transformation-RAG related code

1. **Query Transformation:** We use the LLM to generate 3 alternative versions of the original query.
2. **Multi-Query Retrieval:** We retrieve documents using both the original query and the transformed queries.
3. **Deduplication:** We remove duplicate documents from the retrieved set.
4. **Response Generation:** Using the combined context from all retrieved documents, we generate a final response to the original query.

In [8]:
def query_transformation_rag(original_query, vectorstore, llm):
    # Query Transformation
    transform_prompt = ChatPromptTemplate.from_template(
        "Given the original query, generate 3 alternative versions that might improve "
        "retrieval effectiveness. Each version should capture a different aspect or "
        "use different terminology related to the original query.\n"
        "Original query: {query}\n"
        "Transformed queries (provide 3):"
    )
    transform_chain = transform_prompt | llm
    try:
        transformed_queries_response = transform_chain.invoke({"query": original_query})
        transformed_queries = transformed_queries_response.content.split('\n')
        transformed_queries = [q.split(': ')[-1].strip() for q in transformed_queries if q.strip()]
        transformed_queries = transformed_queries[:3]  # Ensure we have at most 3 queries
    except Exception as e:
        print(f"Error transforming query: {e}")
        transformed_queries = [original_query]  # Fallback to original query

    # Retrieve documents for each transformed query
    all_docs = []
    for query in [original_query] + transformed_queries:
        docs = vectorstore.similarity_search(query, k=2)
        all_docs.extend(docs)

    # Remove duplicates and combine retrieved documents
    unique_docs = list({doc.page_content: doc for doc in all_docs}.values())
    context = "\n\n".join([doc.page_content for doc in unique_docs])

    # Generate response using combined context
    response_prompt = ChatPromptTemplate.from_template(
        "You are an AI assistant tasked with answering questions based on the provided context. "
        "The context contains information retrieved using the original query and its transformed versions. "
        "Please analyze the context carefully and provide a comprehensive answer to the original question. "
        "If the context doesn't contain enough information, use your general knowledge to supplement the answer, "
        "but prioritize information from the context when available.\n\n"
        "Original Question: {original_query}\n"
        "Context:\n{context}\n\n"
        "Answer:"
    )
    response_chain = response_prompt | llm
    try:
        response = response_chain.invoke({"context": context, "original_query": original_query})
        final_answer = response.content
    except Exception as e:
        print(f"Error generating response: {e}")
        final_answer = "I apologize, but I encountered an error while generating the response."

    return {
        "original_query": original_query,
        "transformed_queries": transformed_queries,
        "final_answer": final_answer,
        "retrieved_context": context
    }

### Step 4: Create chunk of web data to Chroma Vector Store

In [ ]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5
)

# Load and process data
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Create vector store
vectorstore = create_vector_store(chunks)

### Step 5: Run Query Transformation RAG

This implementation shows the key parts of Query Transformation RAG:

1. Generation of alternative query formulations to capture different aspects or terminology
2. Retrieval using both original and transformed queries to improve coverage
3. Deduplication of retrieved documents to avoid redundancy
Generation of a comprehensive response using the expanded context

In [10]:
# Example query
original_query = "What are the ethical considerations in AI development?"

# Run Query Transformation RAG
result = query_transformation_rag(original_query, vectorstore, llm)

print("Original Query:", result["original_query"])
print("\nTransformed Queries:")
for i, query in enumerate(result["transformed_queries"], 1):
    print(f"{i}. {query}")
print("\nFinal Answer:")
print(result["final_answer"])
print("\nRetrieved Context (first 500 characters):")
print(result["retrieved_context"][:500] + "...")

Original Query: What are the ethical considerations in AI development?

Transformed Queries:
1. Here are three alternative versions of the original query that might improve retrieval effectiveness:
2. 1. **Transformed query 1:** "What are the moral implications of artificial intelligence in its design and implementation?"
3. This query captures a similar aspect as the original, focusing on the ethical considerations in AI development, but uses more formal and philosophical language.

Final Answer:
Based on the provided context, the ethical considerations in AI development involve testing the permissibility of AI projects at all stages of design, development, and implementation. This requires considering the social and ethical implications of AI systems, as well as collaboration between various job roles. Some key ethical considerations include:

1. Promotion of the wellbeing of people and communities affected by AI technologies.
2. Minimizing risks and ensuring that AI systems make cho